In [2]:
%run SAT_help_functions.ipynb

## OUM

In [3]:
data_dir = "/Users/wangjiashun/Desktop/Reserach on SAT/datasets/Evans2021/Lerche2017/dataBySubject/sub"
sample_size = 38
SAT_conds = ["Speed", "Accuracy"]

In [4]:
class BoundSpeedAcc(Bound):
    name = "BoundSpeedAcc"
    required_parameters = ["Bspeed", "Bacc"]
    required_conditions = ['Cond']
    def get_bound(self, conditions, *args, **kwargs):
        assert self.Bacc > 0
        assert self.Bspeed > 0
        if conditions['Cond'] == 'Speed':
            return self.Bspeed
        elif conditions['Cond'] == 'Accuracy':
            return self.Bacc
        else:
            print("Non-implemented SAT conditions")

In [5]:

class DriftConstantSpeedAcc(Drift):
    name = "DriftConstantSpeedAcc"
    required_parameters = ["Vspeed","Vacc"]
    required_conditions = ['Cond']
        
    def get_drift(self, x, t, conditions, **kwargs):
        if conditions['Cond'] == 'Speed':
            return self.Vspeed
        elif conditions['Cond'] == 'Accuracy':
            return self.Vacc
        else: 
            print("Non-implemented SAT conditions")

In [6]:

class NondecisionSpeedAcc(OverlayNonDecision):
    name = "NondecisionSpeedAcc"
    required_parameters = ["tSpeed", 'tAcc']
    required_conditions = ['Cond']
        
    def get_nondecision_time(self, conditions, **kwargs):
        if conditions['Cond'] == 'Speed':
            return self.tSpeed 
        elif conditions['Cond'] == 'Accuracy':
            return self.tAcc
        else: 
            print("Non-implemented SAT conditions")

In [7]:
def get_DDM_result_sub_2v2a2ter(samp, v = [0,6], a = [0.5,8], ter=[0.1,1], T_dur=5):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftConstantSpeedAcc(Vspeed = Fittable(minval=v[0], maxval=v[1]),
                                                  Vacc = Fittable(minval=v[0], maxval=v[1])), 
                      noise=NoiseConstant(noise=1),
                      bound=BoundSpeedAcc(
                          Bspeed=Fittable(minval=a[0], maxval=a[1]),
                          Bacc=Fittable(minval=a[0], maxval=a[1])),
                      
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),

                      ),
                      dx=.01, dt=.1, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,6),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'Vspeed': 'Drift Rate Speed',
                            'Vacc': 'Drift Rate Accuracy',
                            'Bspeed': 'Boundary Separation Speed',
                            'Bacc': 'Boundary Separation Accuracy',
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [8]:
df_all = pd.DataFrame()
for i in range(sample_size):
    print(i)
    path = os.path.join(data_dir+str(i+1)+".csv")
    df = pd.read_csv(path, sep = ",")
    #df = df[df["cond"]==SAT] # A condition
    df = df[df["Time"]<30]

    samp = Sample.from_pandas_dataframe(df, rt_column_name="Time", correct_column_name="Correct")
    df = get_DDM_result_sub_2v2a2ter(samp, T_dur = 30)
    df_all = pd.concat((df_all,df))
    print(df)
df_OUM = df_all.copy()


0


Info: Params [0.27506277 0.09385408 1.28682439 3.53012233 0.80561345 0.99888283] gave 969.0318701918112


   Drift Rate Speed  Drift Rate Accuracy  Boundary Separation Speed  \
0          0.275063             0.093854                   1.286824   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      3.530122                 0.805613   

   Non-decision Time Accuracy        BIC  
0                    0.998883  969.03187  
1


Info: Params [0.3381974  0.20110414 2.52347299 3.16487693 0.90374874 0.80980379] gave 1042.8672697297945


   Drift Rate Speed  Drift Rate Accuracy  Boundary Separation Speed  \
0          0.338197             0.201104                   2.523473   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      3.164877                 0.903749   

   Non-decision Time Accuracy         BIC  
0                    0.809804  1042.86727  
2


...

   Drift Rate Speed  Drift Rate Accuracy  Boundary Separation Speed  \
0          0.384095             0.228181                   2.486244   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      3.370357                 0.425507   

   Non-decision Time Accuracy          BIC  
0                    0.406442  1066.183504  
36


Info: Params [0.18772712 0.10499073 1.60271348 1.66777595 0.98124846 0.96361996] gave 783.4403940864547


   Drift Rate Speed  Drift Rate Accuracy  Boundary Separation Speed  \
0          0.187727             0.104991                   1.602713   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      1.667776                 0.981248   

   Non-decision Time Accuracy         BIC  
0                     0.96362  783.440394  
37


Info: Params [0.27311557 0.13019094 1.30336187 3.42404081 0.17191472 0.13887109] gave 1068.7679699565356


   Drift Rate Speed  Drift Rate Accuracy  Boundary Separation Speed  \
0          0.273116             0.130191                   1.303362   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      3.424041                 0.171915   

   Non-decision Time Accuracy         BIC  
0                    0.138871  1068.76797  


In [9]:
df_OUM.to_csv("DDM2v2a2terresultLerche2017.csv")

In [10]:
df_OUM["BIC"].mean()

836.3055671329525